In [0]:
import pandas as pd
from databricks import feature_store
from databricks.feature_store import FeatureLookup
import shap # for df
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PandasToSpark").getOrCreate()


# Read data

- SNW connection example.

In [0]:
# SNW example
# options = {
# "sfUrl": "https://dp04782.west-europe.azure.snowflakecomputing.com",
# "sfUser": "es.sgo.saas@cirsa.onmicrosoft.com",
# "sfPassword": "gJu4xkNG7AjNhE1",
# "sfDatabase": "SPACEGO",
# "sfWarehouse": "CONTRIBUTOR_SGO_WH"
# }
# # funciona
# query = f"""
# select top 10 *
# from "SPACEGO"."DEVELOPMENT"."TMP_ERIC_MULTICUENTAS"
# """
# df = spark.read.format("snowflake").options(**options).option("query", query ).load()
# df.head(2)


# table for ML -> convert to Spark df.
df_diabetes = shap.datasets.adult()[0].iloc[:,0:3]
df_diabetes['unique_id'] = df_diabetes.index
int8_columns = df_diabetes.select_dtypes(include='int8').columns
df_diabetes[int8_columns] = df_diabetes[int8_columns].astype('int64')
df_diabetes = spark.createDataFrame(df_diabetes)


In [0]:
# Before creating any feature tables, you must create a database to store them.
database_name = 'folder'
spark.sql(f"CREATE DATABASE IF NOT EXISTS {database_name}")

# Save table in folder
customer_feature_table = feature_store.FeatureStoreClient().create_table(
  name='folder.df_health', primary_keys=['unique_id'],
  schema=df_diabetes.schema, df = df_diabetes, description='Health dataframe'
)

# Now in UI you con go to "Features" and schedule a job!

2024/01/17 10:59:50 INFO databricks.feature_store._compute_client._compute_client: Created feature table 'hive_metastore.folder.df_health'.


In [0]:

load_df = spark.sql("SELECT * FROM folder.df_health")
load_df.show(10)

<FeatureLookup: feature_name=None, lookback_window=None, lookup_key='unique_id', output_name=None, table_name='folder.DiabetesKPI', timestamp_lookup_key=None>